# Correlation matrix and visualization of variables and GDP

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
df= pd.read_csv (os.getcwd()+'\Asia.csv')
df

,Unnamed: 0,Country,Year,Alcohol per capita,Education GExp,Employment-agriculture,Employment-industry,Employment-services,Exports-Commercial services,Exports-G&S,...,Literacy rate,Mortality-infants,Mortality-pollution,Net migration,Ninis,R&D GExp,Renewable electricity,Suicide,Workers high education,Continent
0,126,BGD,1990,0.1180,11.191030,54.253103,14.862414,30.887586,2.956232e+08,1.866930e+09,...,61.730904,359910.00000,149.0,-1739225.0,29.915999,NaN,0.0,4.39,79.083331,Asia
1,127,BGD,1991,0.1180,11.062100,69.510002,13.580000,16.910000,3.442842e+08,2.062577e+09,...,35.319302,345141.00000,149.0,-1739225.0,29.915999,NaN,0.0,4.39,79.083331,Asia
2,128,BGD,1992,0.1180,12.536670,68.900002,12.710000,18.389999,3.926887e+08,2.405650e+09,...,61.730904,330187.00000,149.0,-813794.0,29.915999,NaN,0.0,4.39,79.083331,Asia
3,129,BGD,1993,0.1180,12.113850,68.120003,11.880000,20.000000,4.348718e+08,2.990714e+09,...,61.730904,315347.00000,149.0,-1739225.0,29.915999,NaN,0.0,4.39,79.083331,Asia
4,130,BGD,1994,0.1180,14.903409,67.220001,11.070000,21.709999,4.188037e+08,3.039701e+09,...,61.730904,300680.00000,149.0,-1739225.0,29.915999,NaN,0.0,4.39,79.083331,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,782,KOR,2017,10.0725,NaN,4.920000,25.270000,69.800003,8.872020e+10,6.647312e+11,...,97.965942,1125.00000,20.5,58657.0,NaN,4.29206,669000000.0,25.80,72.300003,Asia
218,783,KOR,2018,9.7000,NaN,5.000000,25.200001,69.800003,1.029451e+11,7.194899e+11,...,97.965942,1059.00000,20.5,92681.5,NaN,4.52753,669000000.0,28.60,72.239998,Asia
219,784,KOR,2019,10.0725,NaN,5.140000,24.580000,70.279999,1.029709e+11,6.486106e+11,...,97.965942,997.00000,20.5,92681.5,NaN,3.03234,669000000.0,28.60,72.300003,Asia
220,785,KOR,2020,10.0725,NaN,8.708621,26.335652,63.290690,8.884410e+10,5.969452e+11,...,97.965942,940.00000,20.5,92681.5,NaN,3.03234,669000000.0,28.05,71.360001,Asia


## Visualization

In [5]:
columns=['Country','Year','Gender equality','Exports-Commercial services','Renewable electricity','Employment-agriculture','Employment-industry','Employment-services','Exports-G&S','Fertility rate','Foreign investment','GDP','Education GExp','Workers high education','Literacy rate','Mortality-pollution','Net migration','Mortality-infants','Health services use','R&D GExp','Ninis','Suicide','International taxes','Alcohol per capita']
clist=df['Country'].unique()

The following loop has been created to test if each variable for each country follows a normal distribution and also to present in an orderly manner all the correlations between the GDP and the variables in an orderly manner.

In [6]:
def multcolumn(frame):
    for i in range(2, len(columns)):
        name2=columns[i]+'^2'
        power_2=df[columns[i]]**2
        name3=columns[i]+'^3'
        power_3=df[columns[i]]**3
        namelog=columns[i]+' log'
        log=np.log(df[columns[i]])
        frame=frame.assign(name2=power_2)
        frame=frame.assign(name3=power_3)
        frame=frame.assign(namelog=log)

In [5]:
for i in range(0,len(clist)):
    dat=df.loc[df.loc[:, 'Country'] == clist[i]]
    for e in range(2,len(columns)):
        data=dat.loc[:, e]
        stat, p = shapiro(data)
        print(clist[i] +"-"+ columns[e])
        print('Statistical=%.3f, p=%.3f' % (stat, p))
        alpha = 0.05
        if p &gt; alpha:
        print('Data is normal ( H0 not denied )')
        else:
        print('It does not seem normal( Denies H0 )')
    dat=multcolumn(dat)   
    corr=dat.corr()
    print(clist[i])
    corr[['GDP']].sort_values(by = 'GDP',ascending = False).style.background_gradient()

SyntaxError: invalid syntax (947940458.py, line 9)

Disposable

In [ ]:
import seaborn as sns
corrmat = corr_df
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)